In [1]:
# importar librerias para el proyecto
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model, tree
from sklearn.feature_selection import RFE
from sklearn import linear_model, tree
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from dtreeviz.trees import dtreeviz
from sklearn.model_selection import GridSearchCV
import joblib as joblib
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
import keras.backend as K
from scikeras.wrappers import KerasClassifier


C:\Users\CARLOS MARIO\AppData\Local\Temp\ipykernel_11924\1027519386.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Cargamos los datos  
datos = pd.read_csv("../Datos/ObesityDataSet_raw_and_data_sinthetic.csv")
datos.head(10)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal_Weight
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal_Weight
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal_Weight
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight


In [41]:
datos.describe()


Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [4]:
rng = np.random.RandomState(seed=42)
#datos["random_cat"] = rng.randint(3, size=datos.shape[0])
#datos["random_num"] = rng.randn(datos.shape[0])
categorical_gender =["Gender"]
categorical_twocat = ["family_history_with_overweight", "FAVC","SMOKE", "SCC"] # Excluí random_cat
categorical_fourcat= ["CAEC","CALC"]
categorical_multicat=['MTRANS']
cat_2=['no','yes']
cat_g=['Female', 'Male']
cat_4=['no', 'Sometimes', 'Frequently', 'Always']
cat_m=['Public_Transportation', 'Walking', 'Automobile', 'Motorbike','Bike']
numerical_columns = ["Age", "FCVC", "NCP", "CH2O", "FAF","TUE"] #Excluí random_num
X = datos[numerical_columns + categorical_gender+categorical_twocat+categorical_fourcat+categorical_multicat]
y = datos["NObeyesdad"]
y = datos['NObeyesdad'].map({'Normal_Weight': 0, 'Insufficient_Weight': 0, 'Overweight_Level_I':1,'Overweight_Level_II':1, 'Obesity_Type_I':2,'Obesity_Type_II':2, 'Obesity_Type_III':2})
yname = ["NObeyesdad"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [43]:
numerical_columns

['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [5]:
X_train.head(10)

,Age,FCVC,NCP,CH2O,FAF,TUE,Gender,family_history_with_overweight,FAVC,SMOKE,SCC,CAEC,CALC,MTRANS
24,21.000000,2.000000,3.000000,2.000000,0.000000,1.000000,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1785,23.912387,2.425503,2.996834,2.017602,1.580263,0.531769,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1187,35.719457,2.000000,1.009426,2.136398,1.060349,0.000000,Male,yes,yes,no,no,Sometimes,no,Automobile
1209,25.929746,3.000000,3.000000,2.045069,1.725931,1.619596,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1965,23.712641,3.000000,3.000000,2.856795,1.046463,0.586163,Female,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1593,23.470070,3.000000,2.701689,2.738485,0.992253,0.000000,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1082,19.126145,2.696381,3.000000,2.472964,1.856119,1.000000,Female,yes,no,no,no,Sometimes,Sometimes,Public_Transportation
1915,26.000000,3.000000,3.000000,2.653563,0.000000,0.554150,Female,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1772,25.012770,2.191108,2.993856,2.444917,1.055854,0.145284,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
655,18.915050,3.000000,3.000000,2.924594,1.352558,0.220087,Female,no,yes,no,yes,Sometimes,Sometimes,Public_Transportation


In [61]:
# Se crea un pipeline para codificar los datos

categorical_encoder1 = OrdinalEncoder(categories=[cat_2])
categorical_encoder2 = OrdinalEncoder(categories=[cat_g])
categorical_encoder3 = OrdinalEncoder(categories=[cat_4])
categorical_encoder4 = OrdinalEncoder(categories=[cat_m])
numerical_pipe = StandardScaler()

preprocessing = ColumnTransformer(
    [
        ("Codificación de Categóricas1", categorical_encoder1, categorical_twocat),
        ("Codificación de Categóricas2", categorical_encoder2, categorical_gender),
        ("Codificación de Categóricas3", categorical_encoder3, categorical_fourcat),
        ("Codificación de Categóricas4", categorical_encoder4, categorical_multicat),
        ("Transformación de numéricas", numerical_pipe, numerical_columns),
    ],
    verbose_feature_names_out=False,
)


In [62]:
modelo = DecisionTreeClassifier(random_state=42)
modelo_arbol = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("clasificador", modelo),  
    ]
)



In [63]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=42)
param_grid = {"Classifier__criterion": ['entropy', 'gini'],
              "Classifier__max_depth": range(3, 5),
              "Classifier__min_samples_leaf": range(2, 10)}

best_tree = GridSearchCV(modelo_arbol, param_grid, cv=k_fold, scoring='accuracy', n_jobs=-1)

In [60]:
best_tree.fit(X_train, y_train)

ValueError: Invalid parameter 'Classifier' for estimator Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('Codificación de '
                                                  'Categóricas1',
                                                  OrdinalEncoder(categories=[['no',
                                                                              'yes']]),
                                                  ['family_history_with_overweight',
                                                   'FAVC', 'SMOKE', 'SCC']),
                                                 ('Codificación de '
                                                  'Categóricas2',
                                                  OrdinalEncoder(categories=[['Female',
                                                                              'Male']]),
                                                  ['Gender']),
                                                 ('Codificació

In [24]:
# Random Forest
modelo=RandomForestClassifier(random_state=42)
modelo_rf = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("clasificador", modelo),
    ]
)

#mejor_gnb= GridSearchCV(modelo_train, param_grid, scoring='accuracy')
#joblib.dump(mejor_gnb,"bm_gnb.gz")

In [32]:
#k_fold = StratifiedKFold(5, shuffle=True, random_state=42)
param_grid = {'classifier__criterion': ['squared_error'],
              'classifier__max_depth': range(3, 10),
              'classifier__min_samples_leaf': range(2, 10)}
#param_grid ={}
best_rf = GridSearchCV(modelo_rf, param_grid, scoring='accuracy', n_jobs=-1)


In [33]:
best_rf.fit(X_train, y_train)

ValueError: Invalid parameter 'classifier' for estimator Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('Codificación de '
                                                  'Categóricas1',
                                                  OrdinalEncoder(categories=[['no',
                                                                              'yes']]),
                                                  ['family_history_with_overweight',
                                                   'FAVC', 'SMOKE', 'SCC']),
                                                 ('Codificación de '
                                                  'Categóricas2',
                                                  OrdinalEncoder(categories=[['Female',
                                                                              'Male']]),
                                                  ['Gender']),
                                                 ('Codificació

In [26]:
gnb2=joblib.load('bm_gnb.gz')
gnb2.fit(X_train, y_train)
#confusion_matrix(y_test, gnb.fit(X_test))
gnb2.best_params_

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\pipeline.py", line 408, in _fit
    X, fitted_transformer

In [ ]:
# modelo_logistico= LogisticRegression()
gnb=GaussianNB()
# arbol=DecisionTreeClassifier(criterion='gini', min_samples_split=100, min_samples_leaf=50)
random_forest = RandomForestClassifier(random_state=42)

def red():
    modelo1 = Sequential()
    modelo1.add(Dense(units=1, input_shape=(16,), activation='sigmoid'))
    modelo1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return modelo1
red1 = red()

modelos=[modelo_logistico,gnb,arbol,random_forest, red()]
resultados=[]
for modelo in modelos:
    nombre_modelo=f'{modelo}.joblib'
    flujo = Pipeline(
        [
            ("Preprocesamiento", preprocessing),
            ("Clasificador", modelo),
        ]
    )
    joblib.dump(flujo, nombre_modelo)
    salida=joblib.dump(flujo, nombre_modelo)
    resultados.append(salida)

from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred))

In [ ]:
resultados

In [ ]:

rf=joblib.load('r_RandomForestClassifier(random_state=42).joblib')
confusion_matrix(y_test, rf.predict(X_test))
rf



In [ ]:
rf.predict(X_test)

In [ ]:
log=joblib.load('r_LogisticRegression().joblib')
confusion_matrix(y_test, log.predict(X_test))

In [ ]:
nb=joblib.load('r_GaussianNB().joblib')
confusion_matrix(y_test, nb.predict(X_test))

In [ ]:
log.fit(X_train,y_train)